### Prompt Templates

Prompt Templates help to turn raw user info to a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system messsage with some custom instructions(but still taking messages as input). Next, we'll add in more input besides just the messages

In [23]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_ollama import ChatOllama
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

model = ChatOllama(model = 'llama3.2')

prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are a helpful assisstant who provides the best answers based on the questions asked to the best of your ability.'),
        MessagesPlaceholder(variable_name='messages')
    ]
)

chain = prompt|model

In [24]:
chain.invoke({'messages':['Hi! My name is Kalyan']})

AIMessage(content="Hello Kalyan! It's nice to meet you. Is there something I can help you with, or would you like to chat for a bit?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-07-22T12:22:55.2907626Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6642737900, 'load_duration': 74140300, 'prompt_eval_count': 57, 'prompt_eval_duration': 2276605900, 'eval_count': 32, 'eval_duration': 4288086500, 'model_name': 'llama3.2'}, id='run--8c7293ef-dfab-45b8-91b2-5bccd9711076-0', usage_metadata={'input_tokens': 57, 'output_tokens': 32, 'total_tokens': 89})

In [25]:
store = {}
def get_chat_session(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

config = {'configurable':{'session_id':'chat1'}}

In [26]:
with_chat_history = RunnableWithMessageHistory(chain, get_chat_session)

In [27]:
response = with_chat_history.invoke("Hi, I am Kalyan!",config=config)
response.content

"Hello Kalyan! It's nice to meet you. Is there something I can help you with today? Do you have a question or topic you'd like to discuss? I'm all ears and here to assist you."

#### Now, we will raise the difficulty by adding multi input variables

In [30]:
gemma_model = ChatOllama(model='gemma3n:e2b')

In [43]:
prompt1 = ChatPromptTemplate.from_messages(
    [
        ('system','You are a helpful assisstant who also knows {language}. Answer the questions to the best of your knowledge only in {language}.'),
        MessagesPlaceholder(variable_name='messages')
    ]
)

chain1 = prompt1|gemma_model

In [32]:
response = chain1.invoke({'messages':['Hi, I am Kalyan and I am studying 3rd year of my Engineering, CSE.'],'language':'Telugu'})
response.content

"నమస్కారం కళ్యాణ్ గారు! మీరు 3వ సంవత్సరం ఇంజనీరింగ్ విద్యార్థిగా CSE చదువుతున్నందుకు సంతోషంగా ఉంది. మీకు ఏమైనా సహాయం కావాలా? ఏదైనా ప్రశ్నలు ఉన్నాయా? నేను మీకు సహాయం చేయడానికి ఇక్కడ ఉన్నాను. (Namaskaram Kalyan garu! Meeru 3va samavaram engineering vidyarthi ga CSE chadavundu nanduku santoshamunga unduni. Meeku edaina sahaayam kaavalaa? Edana prashnulu unnaaya? Nenu meeku sahaayam cheyadaaniki ikkada unnaanu.)\n\n(Translation: Hello Kalyan! It's nice to hear you are a 3rd year Engineering student studying CSE. Do you need any help? Do you have any questions? I am here to help you.)\n\nFeel free to ask anything! I'll do my best to answer in Telugu. Let me know what you'd like to discuss.\n"

*Now let's wrap this complicated chain into the Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.*

In [34]:
with_chat_history1 = RunnableWithMessageHistory(
    chain1,
    get_chat_session,
    input_messages_key='messages'
)

In [44]:
response = with_chat_history1.invoke(
    {'messages':["Hi, I am Kalyan!"],'language':'Hindi'},
    config
)
response.content

'नमस्ते कल्याण! (Namaste Kalyan!)\n\nयह सुनकर बहुत अच्छा लगा कि आप यहाँ हैं! (Yeh sunkar bahut accha laga ki aap yahan hain!)\n\nक्या मैं आपकी किसी चीज़ में मदद कर सकता हूँ? (Kya main aapki kisi cheez mein madad kar sakta hoon?) - Is there anything I can help you with?\n\nकृपया मुझे बताएं कि आप क्या जानना चाहते हैं। (Kripya mujhe bataen ki aap kya jaanna chahte hain.) - Please tell me what you want to know.\n\nमैं आपकी मदद करने के लिए तैयार हूँ! (Main aapki madad karne ke liye taiyaar hoon!) - I am ready to help you!'

In [45]:
response = with_chat_history1.invoke(
    {'messages':["Hey, what's my name?"],'language':'Hindi'},
    config
)
response.content

"नमस्ते! (Namaste!)\n\nआपका नाम कल्याण है! (Aapka naam Kalyan hai!)\n\nThat means your name is Kalyan!  It's nice to meet you, Kalyan!  (यह मिलकर बहुत अच्छा लगा, कल्याण!) - It was nice to meet you, Kalyan!\n\nIs there anything else you'd like to know or talk about? (क्या आप कुछ और जानना या बात करना चाहते हैं?) - Is there anything else you'd like to know or talk about?\n\n\n\n"

In [46]:
response = with_chat_history1.invoke(
    {'messages':["Who am I ?"],'language':'Hindi'},
    config
)
response.content

"यह एक दिलचस्प सवाल है! (Yeh ek dilchasp sawal hai!) - That's an interesting question!\n\nYou are Kalyan! You are asking me questions, and you are communicating with me in Hindi. You are a person who is curious and wants to learn. (आप कल्याण हैं! आप मुझसे सवाल पूछ रहे हैं, और आप मुझसे हिंदी में संवाद कर रहे हैं। आप एक जिज्ञासु व्यक्ति हैं जो सीखना चाहता है।)\n\nEssentially, you are *you* – a person with thoughts, feelings, and the ability to ask questions! (अ essentially, you are *you* – a person with thoughts, feelings, and the ability to ask questions!)\n\nIs there anything *specific* you'd like to know about who you are? (क्या आप *विशिष्ट* रूप से अपने बारे में कुछ जानना चाहते हैं?) - Is there anything *specific* you'd like to know about yourself?\n\n\n\n"

In [50]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am Kalyan!', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Kalyan! It's nice to meet you. Is there something I can help you with today? Do you have a question or topic you'd like to discuss? I'm all ears and here to assist you.", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-07-22T12:23:02.0489612Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6707244100, 'load_duration': 78352300, 'prompt_eval_count': 57, 'prompt_eval_duration': 702956100, 'eval_count': 46, 'eval_duration': 5924731500, 'model_name': 'llama3.2'}, id='run--2007407b-29b4-42e3-86c9-38168c458d10-0', usage_metadata={'input_tokens': 57, 'output_tokens': 46, 'total_tokens': 103}), 'Hi, I am Kalyan!', AIMessage(content="नमस्ते कल्याण! (Namaste Kalyan!) \n\nयह सुनकर बहुत अच्छा लगा कि आप यहाँ हैं! (Yeh sunkar bahut achcha laga ki aap yahan hain!) \n\n(This is great to hear you're here!